<a href="https://colab.research.google.com/github/NadaAdelMousa/NLP_Tweets_Hate_Speech/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 90 Mins**

### Import Libraries

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Load Dataset

In [2]:
!gdown https://drive.google.com/uc?id=1EQCpEpOrVMQquHoEy94BkT4y51nlLUyb -O dataset.csv

Downloading...
From: https://drive.google.com/uc?id=1EQCpEpOrVMQquHoEy94BkT4y51nlLUyb
To: /content/dataset.csv
100% 3.10M/3.10M [00:00<00:00, 56.3MB/s]


In [125]:
df = pd.read_csv('dataset.csv', index_col='id')
df.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation


### EDA

- check NaNs

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31962 entries, 1 to 31962
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   31962 non-null  int64 
 1   tweet   31962 non-null  object
dtypes: int64(1), object(1)
memory usage: 749.1+ KB


- check duplicates

In [127]:
df.duplicated().sum()

2432

In [128]:
df = df.drop_duplicates()

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29530 entries, 1 to 31962
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   29530 non-null  int64 
 1   tweet   29530 non-null  object
dtypes: int64(1), object(1)
memory usage: 692.1+ KB


- show samples of data texts to find out required preprocessing steps

In [130]:
pd.set_option('display.max_colwidth',None)

In [131]:
df.sample(20)

,label,tweet
id,,
18358,0,this week like....
31251,0,9 months later.. i am back to my pre pregnancy weight and a pound less!
16520,0,#motivation #dailyword #life #inspriation #focus #determination #life #sense @ halfway tree
115,1,@user why not @user mocked obama for being black. @user @user @user @user #brexit
8912,0,"â #em's under pressure, focus with brexit - bbh #blog #silver #gold #forex"
24503,0,@user waiting for @user 's update about his trilingual (tamil telugu and hindi) film. #love #june10 #cantwait #exâ¦
9912,0,#bbqðâ #20160611#ktyd#bbq#incident#lol#enjoy #sorry#and#thankyou#love#friend#maâ¦
9576,0,retweet it sweetie ðâ»ï¸ #bbw #curvy #bbws #big #ass #pussy #re #sex #sexy #thick #bbwlover #kik #vip #retweet
21385,0,"are you on if not, check out my next post #designer @user #dribbble"


- check dataset balancing

In [132]:
print(df['label'].value_counts())

0    27517
1     2013
Name: label, dtype: int64


In [106]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = df[(df['label']==0)] 
df_minority = df[(df['label']==1)] 
# upsample minority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample with replacement
                                 n_samples= 2250, # to match majority class
                                 random_state=42)  # reproducible results
                                 
# Combine majority class with upsampled minority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [107]:
df_shuffled = df_downsampled.sample(frac=1, random_state=1).reset_index()

In [108]:
print(df_shuffled['label'].value_counts())

0    2250
1    2013
Name: label, dtype: int64


- Cleaning and Preprocessing are:
    - 1
    - 2
    - 3

### Cleaning and Preprocessing

In [133]:
train_text = df['tweet'].values
labels = df['label'].values

In [12]:
from pathlib import Path
import os
import re
import html
import string
import unicodedata
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [161]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n ").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))

def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
  text = text.replace("'",'')
  text = text.replace("user",'')
  for punc in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~':
    text = text.replace(punc,' ')
  return text


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_repeated_characters(text):
  re1 = re.compile(r'(\S)\1{2,}')
  return re1.sub(' ', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    text = remove_repeated_characters(text)
    text = remove_whitespaces(text)
    words = text2words(text)
    #stop_words = stopwords.words('english')
    #words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem or lemmatize
    #words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return  ''.join(words)

In [162]:
def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [163]:
trn_texts = normalize_corpus(train_text)

In [164]:
for i in range(20):
  print(train_text.tolist()[i]+'\n')
  print(trn_texts[i])
  print('--------------------------------------------------------------------------------')

 @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run

when a father be dysfunctional and be so selfish he drag his kid into his dysfunction run
--------------------------------------------------------------------------------
@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked

thank for lyft credit i cant use cause they dont offer wheelchair vans in pdx disapointed getthanked
--------------------------------------------------------------------------------
  bihday your majesty

bihday your majesty
--------------------------------------------------------------------------------
#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦  

model i love u take with u all the time in ur
--------------------------------------------------------------------------------
 factsguide: society now    #motivation

factsguide society now motivation
-----

In [165]:
print(train_text[0])
print(trn_texts[0])

 @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run
when a father be dysfunctional and be so selfish he drag his kid into his dysfunction run


In [166]:
y = np.asarray(labels).astype('float32')

In [167]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = \
     train_test_split(trn_texts, y, test_size=.2, stratify=y, random_state=42)

In [168]:
print(len(x_train))
print(len(x_test))

23624
5906


**If it takes 60 Mins till here, you are doing Great** <br>
**If not! You also are doing Great**

### Modelling

In [169]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(x_train)

CountVectorizer()

In [170]:
print(vectorizer.get_feature_names_out())
print(len(vectorizer.get_feature_names_out()))

['aa' 'aaberg' 'aaca' ... 'zx' 'zydeco' 'zz']
31274


In [171]:
x_train_bow = vectorizer.transform(x_train)
x_test_bow = vectorizer.transform(x_test)

In [172]:
print(x_train_bow[0].todense())
print(y_train[0])

print(x_test_bow[0].todense())
print(y_test[0])

[[0 0 0 ... 0 0 0]]
0.0
[[0 0 0 ... 0 0 0]]
0.0


In [173]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(x_train_bow, y_train)

LinearSVC()

#### Evaluation

In [174]:
y_pred = clf.predict(x_test_bow)

In [175]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      5573
         1.0       0.61      0.74      0.67       333

    accuracy                           0.96      5906
   macro avg       0.80      0.86      0.82      5906
weighted avg       0.96      0.96      0.96      5906



### Enhancement

- Using different N-grams
- Using different text representation technique

####N-gram(1,2)

In [176]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer.fit(x_train)

CountVectorizer(ngram_range=(1, 2))

In [177]:
print(vectorizer.get_feature_names_out())
print(len(vectorizer.get_feature_names_out()))

['aa' 'aa aa' 'aa aaeac' ... 'zydeco dance' 'zz' 'zz plan']
173476


In [178]:
x_train_bow = vectorizer.transform(x_train)
x_test_bow = vectorizer.transform(x_test)

In [179]:
print(x_train_bow[0].todense())
print(y_train[0])

print(x_test_bow[0].todense())
print(y_test[0])

[[0 0 0 ... 0 0 0]]
0.0
[[0 0 0 ... 0 0 0]]
0.0


In [180]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(x_train_bow, y_train)

LinearSVC()

In [181]:
y_pred = clf.predict(x_test_bow)

In [182]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      5644
         1.0       0.54      0.83      0.65       262

    accuracy                           0.96      5906
   macro avg       0.77      0.90      0.82      5906
weighted avg       0.97      0.96      0.96      5906



####word Embedding

In [36]:
import spacy

In [ ]:
! python -m spacy download en_core_web_md

In [38]:
nlp = spacy.load('en_core_web_md')

In [39]:
x_train_v = np.zeros((len(x_train), 300))
x_test_v = np.zeros((len(x_test), 300))

for i, doc in enumerate(nlp.pipe(x_train)):
    x_train_v[i, :] = doc.vector

for i, doc in enumerate(nlp.pipe(x_test)):
    x_test_v[i, :] = doc.vector

In [40]:
x_train_v[0][:10]

array([-0.68542272,  0.30769712, -0.14465161, -0.01608667, -0.07624113,
       -0.0575876 ,  0.14802013, -0.12496223, -0.01613048,  2.08206344])

In [48]:
from sklearn.svm import LinearSVC

clf = LinearSVC(max_iter=2000)
clf.fit(x_train_v, y_train)

LinearSVC(max_iter=2000)

In [49]:
y_pred = clf.predict(x_test_v)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97      5759
         1.0       0.26      0.72      0.39       147

    accuracy                           0.94      5906
   macro avg       0.63      0.83      0.68      5906
weighted avg       0.97      0.94      0.96      5906



#### Done!